In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
import re 
from spellchecker import SpellChecker
import string
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Pre-Running

In [2]:
# Building a list of Keywords
list_words=['hello','analyze', 'quit']
list_syn={}
for word in list_words:
    synonyms=[]

    # Create synonyms for each Keyword
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():

            # Remove any special characters from synonym strings
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)


print (list_syn['analyze'])

{'analyse', 'analyze', 'study', 'psychoanalyze', 'canvas', 'examine', 'psychoanalyse', 'break down', 'dissect', 'canvass', 'take apart'}


In [3]:
import joblib

# Load the saved classifier
clf_logistic = joblib.load('.\\classifiers\\logistic_regression\\classifier.logistic_regression')
vectorizer_logistic = joblib.load('.\\classifiers\\logistic_regression\\vectors.logistic_regression')
clf_naive = joblib.load('.\\classifiers\\naive_bayes\\classifier.naive_bayes')
vectorizer_naive = joblib.load('.\\classifiers\\naive_bayes\\vectors.naive_bayes')

# Chatbot

In [4]:
# English Stopwords for clean_user_input function
stop_words = set(stopwords.words("english"))
# Spellchecker
spell = SpellChecker()
# Lemmatzier
lemmatizer = WordNetLemmatizer()


def clean_user_input(user_input):
    # Remove HTML tags from user input
    user_input_no_html = re.sub('<.*?>', '', user_input)

    # Word Tokenization
    user_input_token = word_tokenize(user_input_no_html)

    # Lowercase and Remove punctuation
    user_input_punct_lower = [x.lower() for x in user_input_token if x not in punctuation]

    # Remove Numbers
    user_input_no_num = [x for x in user_input_punct_lower if not x.isdigit()]

    # Spellchecker
    correct_words = [spell.correction(word) for word in user_input_no_num]

    # Remove Stopwords
    filtered_words = [word for word in correct_words if word not in stop_words]

    # Lemmatization
    base_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # Join words into sentence
    clean_response = ' '.join(base_words)

    return clean_response

In [5]:
def generate_response(user_input, analyzed_input):
    patterns = {
        r'(?i)({}).*'.format('|'.join(list_syn['hello'])): "Hi, which sentence should i analyze?",
        r'(?i)({}).*'.format('|'.join(list_syn['analyze'])): "I will {}, but with which technique? Logistic Regression or Naive Bayes?".format(clean_user_input(user_input)),
        r'(?i)logistic.*': "{}".format(clf_logistic.predict(vectorizer_logistic.transform([clean_user_input(analyzed_input)]))),
        r'(?i)naive.*': "{}".format(clf_naive.predict(vectorizer_naive.transform([clean_user_input(analyzed_input)]))),
        r'(?i)quit.*': "See You!"
    }

    for pattern, response in patterns.items():
        match = re.match(pattern, user_input)
        if match:
            return response
    return "I'm sorry, but I'm not sure I understand."


def chat():
    print("Hi")
    analyzed_input = ''
    
    while True:
        
        user_input = input("You: ")

        if user_input.lower() == list_syn['analyze']:
            clean_response = clean_user_input(user_input)
            analyzed_input = clean_response
            print(clean_response)

        # Here: Call Function for Logistic Regression or Naive Bayes
        if user_input.lower() == 'logistic':
            if analyzed_input != '':
                print(analyzed_input)
                log_response = clf_logistic.predict(vectorizer_logistic.transform(clean_user_input[analyzed_input]))
                print(log_response)

        if user_input.lower() == 'naive':
            if analyzed_input != '':
                naive_response = clf_naive.predict(vectorizer_naive.transform(clean_user_input[analyzed_input]))
                print(naive_response)
            

        if user_input.lower() == 'quit':
            print("Goodbye!")
            break

        response = generate_response(user_input, analyzed_input)
        print("Mr. C-Bot:", response)

# Example usage
chat()


In [ ]:
clf_naive.predict(vectorizer_naive.transform([clean_user_input("Worst hate never Movie")]))

array([0])